In [ ]:
import numpy as np
import numpy.random as ndm
import torch 
import matplotlib.pyplot as plt
import time
from seed import set_seed
from data_generator import generate_case_Deep
from cpdplm_estimation import CPDPLM
from LFDCP import LFDCP

# data store
F_test_deep = []; G_test_deep = []; Res_test_deep = []
ThetaM = []; etaM = []
Info1_deep = []; Info2_deep = []; Re_F_deep = []; Re_G_deep = []; Re_Res_deep = []
corr = 0.5
Theta = [-1,2]
eta = 2

# hyperparameters
n_layer = 3
n_node = 50
n_epoch = 200
patiences = 20
n_lr = 1.2e-3

# test data
set_seed(14)
test_data = generate_case_Deep(500,corr,Theta,eta)
X_test = test_data['X']
Z_test = test_data['Z']
A_test = test_data['A']
Y_test = test_data['Y']
f_true = test_data['f_X']
g_true = test_data['g_X']
Res_true = test_data['f_X_C']

# train & validation: 4:1
# e.g. n = 1000
n_tr = 800
n_va = 200
n = 1000
# loop 
for b in range(10):   # corresponding to the length of eta
    print('n=', n, 'Loop=', b+1 )
    set_seed(114 + b)

    # Generate data
    train_data = generate_case_Deep(n_tr,corr,Theta,eta)
    val_data = generate_case_Deep(n_va,corr,Theta,eta)

    # Initial values
    Theta0 = [0,1]  #initial theta
    eta0 = np.mean(train_data['Z'])  # initial eta

    
    # Estimation
    res = CPDPLM(train_data,val_data,test_data,Theta0, eta0,\
            n_layer, n_node, n_lr, n_epoch, patiences,show_val=False,maxloop=100,seq=0.01)
      

    # record the results
    Theta_res = res['Theta'] # vector to add row by row
    ThetaM.append(Theta_res)
    eta_res = res['eta'] # change point
    etaM.append(eta_res)
  
    
    # test data to calculate Re and Sd_Re for g and h
    f_test_res = res['f_test'] 
    F_test_deep.append(f_test_res) # vector to add row by row
    g_test_res = res['g_test']
    G_test_deep.append(g_test_res) # vector to add row by row
    Res_test_res = res['f_C_test']
    Res_test_deep.append(Res_test_res) # vector to add row by row
    Re_F_deep.append(np.sqrt(np.mean((f_test_res-np.mean(f_test_res)-f_true)**2)/np.mean(f_true**2))) #Re loss of g(x)
    Re_G_deep.append(np.sqrt(np.mean((g_test_res-np.mean(g_test_res)-g_true)**2)/np.mean(g_true**2)))
    Re_Res_deep.append(np.sqrt(np.mean((Res_test_res-np.mean(Res_test_res)-Res_true)**2)/np.mean(Res_true**2)))

    # Calculataion of the score and information
    f_C_train = res['f_C_train']
    f_C_val = res['f_C_val']
    Z_train = train_data['Z']
    Z_val = val_data['Z']
    A1_train = train_data['A']
    A1_val = val_data['A']
    A2_train = train_data['A']*(Z_train>eta)
    A2_val = val_data['A']*(Z_val>eta)
    ## LFD for beta and gamma respectively, note as a1 and a2
    a1 = LFDCP(A1_train, A1_val, train_data,val_data,Theta_res,eta_res,f_C_train, f_C_val,\
            n_layer=3,n_node=50,n_lr=2e-3,n_epoch=200,patiences=10,show_val = False)
    a2 = LFDCP(A2_train, A2_val, train_data,val_data,Theta_res,eta_res,f_C_train, f_C_val,\
            n_layer=3,n_node=50,n_lr=2e-3,n_epoch=200,patiences=10,show_val = False)
    Info = np.zeros((2,2))
    Y_train = train_data['Y']
    X_train = train_data['X']
    A_train = train_data['A']
    AC_train = np.vstack((A_train,A_train*(Z_train>eta)))
    AC_train = AC_train.T
    epsilon_train = Y_train - AC_train@Theta_res -f_C_train
    Info[0,0] = np.mean(epsilon_train**2*(A1_train-a1)**2)
    Info[1,1] = np.mean(epsilon_train**2*(A2_train-a2)**2)
    Info[0,1] = np.mean(epsilon_train**2*(A1_train-a1)*(A2_train-a2))
    Info[1,0] = Info[0,1]
    Sigma = np.linalg.inv(Info)/n_tr
    se1 = np.sqrt(Sigma[0,0])
    Info1_deep.append(se1)
    se2 = np.sqrt(Sigma[1,1])
    Info2_deep.append(se2)
#     time.sleep(1)


#Error_g_dcp = np.mean(np.array(G_test_deep), axis=0) - g_true
#Error_h_dcp = np.mean(np.array(H_test_deep), axis=0) - h_true
#Error_Res_dcp = np.mean(np.array(Res_test_deep), axis=0) - Res_true
Theta_dcp = np.mean(np.array(ThetaM),axis=0)
eta_dcp = np.mean(np.array(etaM))
Sd_F_deep = (np.sqrt(np.mean((Re_F_deep-np.mean(Re_F_deep))**2)))
Sd_G_deep = (np.sqrt(np.mean((Re_G_deep-np.mean(Re_G_deep))**2)))
Sd_Res_deep = (np.sqrt(np.mean((Re_Res_deep-np.mean(Re_Res_deep))**2)))
ThetaM1 = np.array(ThetaM)[:,0]
ThetaM2 = np.array(ThetaM)[:,1]
Bias1_deep = (np.mean(np.array(ThetaM1))-Theta[0])
Sse1_deep = (np.sqrt(np.mean((np.array(ThetaM1)-np.mean(np.array(ThetaM1)))**2)))
Ese1_deep = (np.mean(np.array(Info1_deep)))
Cp1_deep = (np.mean((np.array(ThetaM1)-1.96*np.array(Info1_deep)<=Theta[0])*\
                       (Theta[0]<=np.array(ThetaM1)+1.96*np.array(Info1_deep))))
Bias2_deep = (np.mean(np.array(ThetaM2))-Theta[1])
Sse2_deep = (np.sqrt(np.mean((np.array(ThetaM2)-np.mean(np.array(ThetaM2)))**2)))
Ese2_deep = (np.mean(np.array(Info2_deep)))
Cp2_deep = (np.mean((np.array(ThetaM2)-1.96*np.array(Info2_deep)<=Theta[1])*\
                       (Theta[1]<=np.array(ThetaM2)+1.96*np.array(Info2_deep))))
Bias_eta = np.mean(np.array(etaM))-eta
Sd_eta = np.mean(np.sqrt((np.array(etaM)-np.mean(np.array(etaM)))**2))
etaMS = np.sort(np.array(etaM))
Length_eta = etaMS[9]-etaMS[1]  #95% interval's length
#print(Error_g_dcp)
#print(Error_h_dcp)
#print(Error_Res_dcp)
print('Estimation for reg para and change point para: ')
print('Theta: ', Theta_dcp)
print('eta: ', eta_dcp)
print('Estimation for Re and Sd of deep function: ')
print('Re_g: ', np.mean(Re_F_deep))
print('Re_h: ', np.mean(Re_G_deep))
print('Re_total: ', np.mean(Re_Res_deep))
print('SdRe_g: ', Sd_F_deep)
print('SdRe_h: ', Sd_G_deep)
print('SdRe_total: ', Sd_Res_deep)
print('Inference for beta: ')
print('Bias1: ', Bias1_deep)
print('Sse1: ', Sse1_deep)
print('Ese1: ', Ese1_deep)
print('Cp1: ', Cp1_deep)
print('Inference for gamma: ')
print('Bias2: ', Bias2_deep)
print('Sse2: ', Sse2_deep)
print('Ese2: ', Ese2_deep)
print('Cp2: ', Cp2_deep)
print('Inference for eta: ')
print('Bias_eta: ', Bias_eta)
print('Length_eta: ', Length_eta)
print('Sd_eta: ', Sd_eta)

n= 1000 Loop= 1
n= 1000 Loop= 2
n= 1000 Loop= 3
n= 1000 Loop= 4
n= 1000 Loop= 5
n= 1000 Loop= 6
n= 1000 Loop= 7
n= 1000 Loop= 8
n= 1000 Loop= 9
n= 1000 Loop= 10
Estimation for reg para and change point para:
Theta:  [-0.99396505  2.05899697]
eta:  2.0019996
Estimation for Re and Sd of deep function:
Re_g:  0.14533289
Re_h:  0.17583993
Re_total:  0.11999215
SdRe_g:  0.011429506
SdRe_h:  0.019241247
SdRe_total:  0.01081113
Inference for beta:
Bias1: 0.006034949869911932
Sse1:  0.0868563505213218
Ese1:  0.08937937821334507
Cp1:  1.0
Inference for gamma:
Bias2: 0.05899696667812826
Sse2:  0.11887059885784668
Ese2:  0.10493998647416211
Cp2:  0.8
Inference for zeta:
Bias_eta: 0.0019996166
Length_eta 0.00999999
Sd_eta 0.0048000338
